# Efficient estimation techniques for Variational Quantum Simulation

In [ ]:
!pip install pulser==0.17.3

## Variational Quantum Simulation for the $H_2$ molecule

The main problem with usual variational classical algorithms, the classical counterparts of VQS, is computing the value of the $2^n \times 2^n$ matrix on the output state vector $\bra{\psi}H\ket{\psi}$ after each loop of the algorithm, which grows exponentially in the size of the system. The purpose of VQS algorithms is to offer a solution which time complexity only grows polynomially, thanks to reading all the important properties on the quantum state. Therefore, we need accurate and efficient methods to estimate these properties, which we'll present afterwards.

For now, let's focus on what makes a VQS algorithm, specifically for computing the groundstate energy of the $H_2$ molecule.

### Jordan-Wigner Hamiltonian (cost function)

We need to write the Hamiltonian in a way that's compatible with the formalism of quantum computing. We first second-quantize the Hamiltonian, obtaining an expression in terms of fermionic operators $a, a^\dagger$. Then, we use the Jordan-Wigner transformation, which maps the fermionic operators to Pauli matrices. We obtain the Hamiltonian below, acting on $4$ qubits, decomposed in terms of the coefficients in front of the Pauli matrices.

[This article by Seeley et al.](https://math.berkeley.edu/~linlin/2018Spring_290/SRL12.pdf) gives us the value of 
$H_{JW}$.

$$H_{J W}=-0.81261 \mathbb{1}+0.171201 \sigma_{0}^{z}+0.171201 \sigma_{1}^{z}-0.2227965 \sigma_{2}^{z} \\
-0.2227965 \sigma_{3}^{z} +0.16862325 \sigma_{1}^{z} \sigma_{0}^{z}+0.12054625 \sigma_{2}^{z} \sigma_{0}^{z} \\
+0.165868 \sigma_{2}^{z} \sigma_{1}^{z}+0.165868 \sigma_{3}^{z} \sigma_{0}^{z} +0.12054625 \sigma_{3}^{z}\sigma_{1}^{z} \\
+0.17434925 \sigma_{3}^{z} \sigma_{2}^{z}-0.04532175 \sigma_{3}^{x} \sigma_{2}^{x} \sigma_{1}^{y} \sigma_{0}^{y}\\
+0.04532175 \sigma_{3}^{x} \sigma_{2}^{y} \sigma_{1}^{y} \sigma_{0}^{x}+0.04532175 \sigma_{3}^{y} \sigma_{2}^{x}
\sigma_{1}^{x} \sigma_{0}^{y} -0.04532175 \sigma_{3}^{y} \sigma_{2}^{y} \sigma_{1}^{x} \sigma_{0}^{x}$$

In [ ]:
import numpy as np
import qutip
import matplotlib.pyplot as plt
from scipy.optimize import minimize

from pulser import Register, Sequence, Pulse
from pulser.devices import Chadoq2
from pulser_simulation import Simulation

In [ ]:
num_qubits = 4
zero_state = qutip.basis(2, 0).proj()
one_state = qutip.basis(2, 1).proj()
hadamard = 1 / np.sqrt(2) * qutip.Qobj([[1.0, 1.0], [1.0, -1.0]])
h_mul_phase = qutip.Qobj(np.array([[1.0, 1], [1.0j, -1.0j]])) / np.sqrt(2)
unitary_ensemble = [hadamard, h_mul_phase, qutip.qeye(2)]

g = qutip.basis(2, 1)
r = qutip.basis(2, 0)
n = r * r.dag()

sx = qutip.sigmax()
sy = qutip.sigmay()
sz = qutip.sigmaz()

gggg = qutip.tensor([g, g, g, g])
ggrr = qutip.tensor([g, g, r, r])

In [ ]:
def pauli(positions=[], operators=[]):
    op_list = [
        operators[positions.index(j)] if j in positions else qutip.qeye(2)
        for j in range(num_qubits)
    ]
    return qutip.tensor(op_list)

In [ ]:
coeff_fact = [
    0.81261,
    0.171201,
    0.2227965,
    0.16862325,
    0.174349,
    0.12054625,
    0.165868,
    0.04532175,
]

paulis = [
    pauli(),
    pauli([0], [sz]) + pauli([1], [sz]),
    pauli([2], [sz]) + pauli([3], [sz]),
    pauli([1, 0], [sz, sz]),
    pauli([3, 2], [sz, sz]),
    pauli([2, 0], [sz, sz]) + pauli([3, 1], [sz, sz]),
    pauli([2, 1], [sz, sz]) + pauli([3, 0], [sz, sz]),
    pauli([3, 2, 1, 0], [sx, sx, sy, sy])
    + pauli([3, 2, 1, 0], [sy, sy, sx, sx]),
    pauli([3, 2, 1, 0], [sx, sy, sy, sx])
    + pauli([3, 2, 1, 0], [sy, sx, sx, sy]),
]

In [ ]:
# H2 Molecule : 4 qubits in Jordan-Wigner mapping of the Hamiltonian
a = 10
reg = Register.from_coordinates(
    [
        [0, 0],
        [a, 0],
        [0.5 * a, a * np.sqrt(3) / 2],
        [0.5 * a, -a * np.sqrt(3) / 2],
    ]
)
reg.draw()

Let us keep the exact ground-state energy of the molecule for future reference, by diagonalizing it exactly - this is possible for such a small system, however, this quickly becomes an intractable problem for large molecules.

In [ ]:
def cost_hamiltonian_JW():
    H = (
        -coeff_fact[0] * paulis[0]
        + coeff_fact[1] * paulis[1]
        - coeff_fact[2] * paulis[2]
        + coeff_fact[3] * paulis[3]
        + coeff_fact[4] * paulis[4]
        + coeff_fact[5] * paulis[5]
        + coeff_fact[6] * paulis[6]
        - coeff_fact[7] * paulis[7]
        + coeff_fact[7] * paulis[8]
    )
    return H


global H
H = cost_hamiltonian_JW()
exact_energy, ground_state = H.groundstate()
print(exact_energy, ground_state)

### Quantum Loop (VQS)

Much like in the *Using QAOA to solve a QUBO problem* notebook, we will use a mixed classical-quantum approach for minimizing the energy. The quantum part will do the exploration in Hilbert space, according to a certain set of parameters $\theta_i, \tau_j$, and the classical part will find the optimal parameters given the value of the energy after each loop. For now, we will ignore sampling problems and simply compute the exact expectation value of $H_{JW}$. See [this article by Xiao Yuan et al.](https://arxiv.org/abs/1812.08767) for details about VQS algorithms.

Two mixing Hamiltonians are used for the exploration of the solution space :
$H_1 = \hbar / 2 \sum_i \sigma_i^x + \sum_{j<i}\frac{C_6}{|\textbf{r}_i-\textbf{r}_j|^{6}} \hat n_i \hat n_j$ and $H_2 = H_1 + \hbar / 2 \sum_i \sigma_i^z$.
We apply them repeatedly one after the other in $p$ layers. In total, $2p$ unitaries $U(\theta_i, H_1) = \exp(-i \theta_i H_1)$ and $U(\tau_i, H_2) = \exp(-i \tau_i H_2)$ act on the initial state to produce state $|\Psi(\theta, \tau)\rangle$ and measure $H_{JW}$.

In [ ]:
def quantum_loop(param, in_state, r=reg):
    """
    Args:
        param (np.array): time parameters for each mixing Hamiltonian. There are 2p time parameters in param.
        in_state (qubit.Qobj): initial state.
    """
    seq = Sequence(r, Chadoq2)
    seq.declare_channel("ch0", "rydberg_global")
    middle = len(param) // 2

    for tau, t in zip(param[middle:], param[:middle]):
        pulse_1 = Pulse.ConstantPulse(tau, 1.0, 0, 0)
        pulse_2 = Pulse.ConstantPulse(t, 1.0, 1.0, 0)
        seq.add(pulse_1, "ch0")
        seq.add(pulse_2, "ch0")

    seq.measure("ground-rydberg")
    simul = Simulation(seq, sampling_rate=0.05)
    simul.initial_state = in_state
    results = simul.run()
    return results.expect([H])[-1][-1]


def loop_JW(param, in_state):
    res = minimize(
        quantum_loop,
        param,
        method="Nelder-Mead",
        args=in_state,
        options={"return_all": True, "maxiter": 50, "adaptive": True},
    )
    return res

We choose to act on the quantum states with $5$ layers of noncommuting mixing Hamiltonians, and an initial set of parameters such that pulses with Hamiltonian $H_1$ last $2\mu s$, and those with $H_2$ last $4\mu s$.

In [ ]:
# Setup for VQS
layers = 5
param = [2000] * layers + [4000] * layers

We now obtain the ground-state energy :

In [ ]:
import warnings

# Ignore the warnings
warnings.filterwarnings("ignore", category=UserWarning)

loop_ising_results = loop_JW(param, gggg)
print(loop_ising_results.fun, exact_energy)

As we can see, it's not so far off, since we're about $2$% off from the exact value. Adding more layers, tweaking the mixing Hamiltonians or the initial parameters can help with the accuracy. 

Let's see how well the optimizer did after each loop.

In [ ]:
plt.plot(
    [quantum_loop(pars, gggg) for pars in loop_ising_results.allvecs], "k"
)
plt.axhline(exact_energy, color="red")

Seems like we can cut on calculation time by only allowing $100$ iterations, since we don't get much more accurate afterwards.